In [23]:
SPLICEBERT_PATH = "C:/Users/radek/Downloads/models.tar/models/models/SpliceBERT.1024nt" # this has to be changed, model weigths are avaiable on the github page
import torch
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM, AutoModelForTokenClassification,AutoModelForSequenceClassification,BertForSequenceClassification


In [18]:
tokenizer = AutoTokenizer.from_pretrained(SPLICEBERT_PATH)

In [19]:
seq = "ACGUACGuacguaCGu"  ## WARNING: this is just a demo. SpliceBERT may not work on sequences shorter than 64nt as it was trained on sequences of 64-1024nt in length
seq = ' '.join(list(seq.upper().replace("U", "T"))) # U -> T and add whitespace
input_ids = tokenizer.encode(seq) # N -> 5, A -> 6, C -> 7, G -> 8, T(U) -> 9. NOTE: a [CLS] and a [SEP] token will be added to the start and the end of seq
input_ids = torch.as_tensor(input_ids) # convert python list to Tensor
input_ids = input_ids.unsqueeze(0) # add batch dimension, shape: (batch_size, sequence_length)

In [24]:
#model for classification cancerous or not, but without the fine tuning yet
model = BertForSequenceClassification.from_pretrained(
    "C:/Users/radek/Downloads/models.tar/models/models/SpliceBERT.1024nt",
    num_labels=2  # binary classification
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at C:/Users/radek/Downloads/models.tar/models/models/SpliceBERT.1024nt and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
#tokenizing the data, BERT model runs on tokenized data. 
def kmer_tokenize(sequence, k=3):
    return ' '.join([sequence[i:i+k] for i in range(len(sequence) - k + 1)])

In [26]:
from torch.utils.data import Dataset
import torch

class DNADataset(Dataset):
    def __init__(self, sequences, labels, tokenizer, k=3):
        self.sequences = sequences
        self.labels = labels
        self.tokenizer = tokenizer
        self.k = k

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        sequence = self.sequences[idx]
        kmer_seq = ' '.join([sequence[i:i+self.k] for i in range(len(sequence) - self.k + 1)])
        inputs = self.tokenizer(kmer_seq, padding='max_length', truncation=True, max_length=512, return_tensors="pt")
        item = {key: val.squeeze(0) for key, val in inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item